# Worksheet 4: Aggregations and ACID transaction in MongoDB

## Establish Pymongo connection

Recall that in worksheet1, we have created a cluster on MongoDB Atlas and connect to it via Pymongo and a credentials JSON file. Let's reestablish that connection for our exercises.

- Make sure you use the `adsc_3610` conda environment.
- You might need to copy & paste the `credentials_mongodb.json` file that you used in worksheet1 to the working directory of worksheet3.

In [1]:
from pymongo import MongoClient # import mongo client to connect
import json # import json to load credentials
import urllib.parse

# load credentials from json file
with open('credentials_mongodb.json') as f:
    login = json.load(f)

# assign credentials to variables
username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)

In [2]:
# connect to the database
client = MongoClient(url)

In [3]:
# drop database books and students if they exist
client.drop_database('bookstore')
client.drop_database('students')

To test if your connection has been succesful, let's try to print out all the databases

In [4]:
# list all databases
client.list_database_names()

['banking',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'shop',
 'admin',
 'local']

- Load `sample_mflix` database into an object called db
- Load the `movies` collection into an object called `collection`

In [5]:
db = client['sample_mflix'] # SOLUTION
collection = db['movies'] # SOLUTION

Print out the first document

In [6]:
collection.find_one() # SOLUTION


{'_id': ObjectId('573a1390f29313caabcd50e5'),
 'plot': 'The cartoonist, Winsor McCay, brings the Dinosaurus back to life in the figure of his latest creation, Gertie the Dinosaur.',
 'genres': ['Animation', 'Short', 'Comedy'],
 'runtime': 12,
 'cast': ['Winsor McCay', 'George McManus', 'Roy L. McCardell'],
 'num_mflix_comments': 0,
 'poster': 'https://m.media-amazon.com/images/M/MV5BMTQxNzI4ODQ3NF5BMl5BanBnXkFtZTgwNzY5NzMwMjE@._V1_SY1000_SX677_AL_.jpg',
 'title': 'Gertie the Dinosaur',
 'fullplot': 'Winsor Z. McCay bets another cartoonist that he can animate a dinosaur. So he draws a big friendly herbivore called Gertie. Then he get into his own picture. Gertie walks through the picture, eats a tree, meets her creator, and takes him carefully on her back for a ride.',
 'languages': ['English'],
 'released': datetime.datetime(1914, 9, 15, 0, 0),
 'directors': ['Winsor McCay'],
 'writers': ['Winsor McCay'],
 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'},
 'lastupdated': '2015

In [7]:
# assert db is a database refering to sample_mflix
assert db.name == 'sample_mflix', f"expected db name to be 'sample_mflix' but got {db.name}"

# Assert that the collection names match, ignoring order
assert set(db.list_collection_names()) == set(['sessions', 'comments', 'movies', 'theaters', 'users', 'embedded_movies'])

# assert collection is a collection refering to movies
assert collection.name == 'movies', f"expected collection name to be 'movies' but got {collection.name}"

# assert collection count documents
assert collection.count_documents({}) == 21349, f"expected 21349 documents but got {collection.count_documents({})}"

#### Exercise 1: Calculate the Average Rating of Movies by Genre

rubric: {accuracy = 2}

**Task**: Write an aggregation pipeline to calculate the average imdb rating of movies grouped by genre.

**Hint**: First, use `$unwind` stage to unwind the `genres` array before grouping by genre. Use the `$group` stage to group by genre and the `$avg` operator to calculate the average rating.



Expected output should look something like this:

```json
{'_id': 'War', 'averageRating': 7.128591954022989}
{'_id': 'News', 'averageRating': 7.252272727272728}
{'_id': 'Romance', 'averageRating': 6.6564272782136396}
...
{'_id': 'Sci-Fi', 'averageRating': 6.123609653725079}
{'_id': 'Film-Noir', 'averageRating': 7.397402597402598}
```

In [8]:
# Exercise 1: Calculate the Average Rating of Movies by Genre

pipeline1 = [
    # BEGIN SOLUTION
    {
        "$unwind": "$genres"
    },
    {
        "$group": {
            "_id": "$genres",
            "averageRating": { "$avg": "$imdb.rating" }
        }
    }
    # END SOLUTION
]

# Execute the pipeline
result1 = list(collection.aggregate(pipeline1))
for doc in result1:
    print(doc)

{'_id': 'Documentary', 'averageRating': 7.365679824561403}
{'_id': 'Drama', 'averageRating': 6.803377338624768}
{'_id': 'Comedy', 'averageRating': 6.450214658080344}
{'_id': 'Family', 'averageRating': 6.3296712109061755}
{'_id': 'Action', 'averageRating': 6.347098402018503}
{'_id': 'Sport', 'averageRating': 6.749041095890411}
{'_id': 'Horror', 'averageRating': 5.784709897610922}
{'_id': 'Film-Noir', 'averageRating': 7.397402597402598}
{'_id': 'Talk-Show', 'averageRating': 7.0}
{'_id': 'Western', 'averageRating': 6.823553719008264}
{'_id': 'Short', 'averageRating': 7.377574370709382}
{'_id': 'Adventure', 'averageRating': 6.493680884676145}
{'_id': 'Mystery', 'averageRating': 6.527425044091711}
{'_id': 'Thriller', 'averageRating': 6.304498977505112}
{'_id': 'Music', 'averageRating': 6.883333333333334}
{'_id': 'Fantasy', 'averageRating': 6.3829847908745245}
{'_id': 'Crime', 'averageRating': 6.688585405625764}
{'_id': 'Sci-Fi', 'averageRating': 6.123609653725079}
{'_id': 'News', 'averageRa

In [9]:
import numpy as np

# Filter for the genre 'Mystery' and extract the average rating
mystery_rating = list(filter(lambda x: x['_id'] == 'Mystery', result1))[0]['averageRating']

# Use np.isclose to compare the average rating
assert np.isclose(mystery_rating, 6.527425044091, atol=0.01), \
    f"expected average rating close to 6.53 but got {mystery_rating}"



#### Exercise 2: Find the Top 5 Movies with the Highest IMDb Rating

rubric: {accuracy = 3}

**Task**: Write an aggregation pipeline to find the top 5 movies with the highest Tomatoes Viewer rating. In the result, only keep the movies `title` and `tomatoes.viewer.rating` field only, remove `_id` from the result.

**Hint**: Use the `$sort` stage to sort by Tomatoes Viewer rating in descending order and the `$limit` stage to limit the results to the top 5. Use `$project` to select fields that you want to include/exclude in the output.



Expected output:

```json
...
{'title': 'Murder by Natural Causes', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'I Am Maria', 'tomatoes': {'viewer': {'rating': 5.0}}}
```

In [10]:
# print out tomatoes viewer rating of some movies
for doc in collection.find({}, {"_id": 0,
                                "title": 1, 
                                "tomatoes.viewer.rating": 1}).limit(5):
    print(doc)

{'title': 'Gertie the Dinosaur', 'tomatoes': {'viewer': {'rating': 3.7}}}
{'title': 'The Great Train Robbery', 'tomatoes': {'viewer': {'rating': 3.7}}}
{'title': 'The Italian', 'tomatoes': {'viewer': {'rating': 4.0}}}
{'title': 'Civilization', 'tomatoes': {'viewer': {'rating': 0.0}}}
{'title': 'Where Are My Children?', 'tomatoes': {'viewer': {'rating': 3.1}}}


In [11]:
# Exercise 2: Find the Top 5 Movies with the Highest Tomatoes Viewer Rating

pipeline2 = [
    # BEGIN SOLUTION
    {
        "$sort": {
            "tomatoes.viewer.rating": -1
        }
    },
    {
        "$limit": 5
    },
    {
        "$project": {
            "_id": 0,
            "title": 1,
            "tomatoes.viewer.rating": 1
        }
    }
    # END SOLUTION
]

# Execute the pipeline
result2 = list(collection.aggregate(pipeline2))
for doc in result2:
    print(doc)

{'title': 'Kadin Hamlet', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'The Seine Meets Paris', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'Night Function', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'Murder by Natural Causes', 'tomatoes': {'viewer': {'rating': 5.0}}}
{'title': 'I Am Maria', 'tomatoes': {'viewer': {'rating': 5.0}}}


In [12]:
# check if the result2 has 5 documents
assert len(result2) == 5, f"Expected 5 documents, but found {len(result2)}"

# check if the result2 has the correct keys
assert set(result2[0].keys()) == set(['title', 'tomatoes']), f"Expected keys: ['title', 'tomatoes'], but found {result2[0].keys()}"

# Check if result2 has the correct titles
expected_titles = [
    'Kadin Hamlet',
    'The Seine Meets Paris',
    'Night Function',
    'Murder by Natural Causes',
    'I Am Maria'
]
result_titles = [doc['title'] for doc in result2]
assert set(result_titles) == set(expected_titles), f"Expected titles: {expected_titles}, but found: {result_titles}"



#### Exercise 3: Count the Number of Movies by Year

rubric: {accuracy = 2}

**Task**: Write an aggregation pipeline to count the number of movies released each year. Sort by count in descending order

**Hint**: Use the `$group` stage to group by year and the `$count` operator to count the number of movies. Use the `sort` stage to sort by count



Expected output:
```json
...
{'_id': 2008, 'count': 886}
{'_id': 2010, 'count': 866}
...
```

In [13]:
# Exercise 3: Count the Number of Movies by Year

pipeline3 = [
    # BEGIN SOLUTION
    {
        "$group": {
            "_id": "$year",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": {
            "count": -1
        }
    }
    # END SOLUTION
]

# Execute the pipeline
result3 = list(collection.aggregate(pipeline3))
for doc in result3:
    print(doc)

{'_id': 2013, 'count': 1105}
{'_id': 2014, 'count': 1073}
{'_id': 2012, 'count': 955}
{'_id': 2009, 'count': 917}
{'_id': 2011, 'count': 893}
{'_id': 2008, 'count': 886}
{'_id': 2010, 'count': 866}
{'_id': 2007, 'count': 810}
{'_id': 2006, 'count': 774}
{'_id': 2005, 'count': 713}
{'_id': 2004, 'count': 678}
{'_id': 2002, 'count': 622}
{'_id': 2001, 'count': 612}
{'_id': 2003, 'count': 603}
{'_id': 2000, 'count': 581}
{'_id': 1999, 'count': 515}
{'_id': 1998, 'count': 513}
{'_id': 2015, 'count': 480}
{'_id': 1997, 'count': 439}
{'_id': 1996, 'count': 407}
{'_id': 1995, 'count': 372}
{'_id': 1994, 'count': 305}
{'_id': 1993, 'count': 274}
{'_id': 1992, 'count': 270}
{'_id': 1988, 'count': 251}
{'_id': 1991, 'count': 238}
{'_id': 1989, 'count': 232}
{'_id': 1990, 'count': 225}
{'_id': 1987, 'count': 222}
{'_id': 1984, 'count': 199}
{'_id': 1986, 'count': 190}
{'_id': 1985, 'count': 189}
{'_id': 1982, 'count': 177}
{'_id': 1981, 'count': 168}
{'_id': 1980, 'count': 167}
{'_id': 1983, 'cou

In [14]:
# Filter for the year 2013 and extract the count
count_2013 = list(filter(lambda x: x['_id'] == 2013, result3))[0]['count']

# Assert the count for the year 2013
assert count_2013 == 1105, f"expected count 1105 for the year 2013 but got {count_2013}"

In [15]:
# Filter for the year 2014 and extract the count
count_2014 = list(filter(lambda x: x['_id'] == 2014, result3))[0]['count']

# Assert the count for the year 2014
assert count_2014 == 1073, f"expected count 1073 for the year 2014 but got {count_2014}"



#### Exercise 4: Find the Top 3 Directors with the Most Movies

rubric: {accuracy = 4}

**Task**: Write an aggregation pipeline to find the top 3 directors who have directed the most movies.

**Hint**: Use the `$group` stage to group by director, the `$count` operator to count the number of movies, the `$sort` stage to sort by the count in descending order, and the `$limit` stage to limit the results to the top 3.

You might need to `$unwind` the directors array.



Expected output:
```json
{'_id': ..., 'movieCount': ...}
{'_id': ..., 'movieCount': ...}
{'_id': ..., 'movieCount': ...}
```

In [16]:
# Exercise 4: Find the Top 3 Directors with the Most Movies

pipeline4 = [
    # BEGIN SOLUTION
    {
        "$unwind": "$directors" # unwind the directors array
    },
    {
        "$group": {
            "_id": "$directors",
            "movieCount": { "$sum": 1 }
        }
    },
    {
        "$sort": {
            "movieCount": -1
        }
    },
    {
        "$limit": 3
    }
    # END SOLUTION
]

# Execute the pipeline
result4 = list(collection.aggregate(pipeline4))
for doc in result4:
    print(doc)

{'_id': 'Woody Allen', 'movieCount': 40}
{'_id': 'Martin Scorsese', 'movieCount': 32}
{'_id': 'Takashi Miike', 'movieCount': 31}


In [17]:
# assert Woody Allen, Martin Scorsese, Takashi Miike are the top 3 directors
assert set([doc['_id'] for doc in result4]) == set(['Woody Allen', 'Martin Scorsese', 'Takashi Miike']), \
    f"expected directors: ['Woody Allen', 'Martin Scorsese', 'Takashi Miike'], but got {[doc['_id'] for doc in result4]}"

In [18]:
# assert the movie count for Woody Allen is 40
assert list(filter(lambda x: x['_id'] == 'Woody Allen', result4))[0]['movieCount'] == 40, \
    f"expected Woody Allen to have 40 movies, but got {list(filter(lambda x: x['_id'] == 'Woody Allen', result4))[0]['movieCount']}"

### Exercise 5: Calculate the Average IMDb Rating and Number of Movies for Each Genre (Rounded to 2 Decimals)

rubric: {accuracy = 3}

**Task**: Write an aggregation pipeline to calculate the average IMDb rating (rounded to 2 decimals) and the number of movies for each genre.

**Hint**: Use the `$unwind` stage to deconstruct the genres array, the `$group` stage to group by genre, the `$avg` operator to calculate the average rating, and the `$project` stage to round the average rating to 2 decimal places.



Expected output:

```json
...
{'_id': 'Crime', 'movieCount': 2457, 'averageRating': 6.69}
{'_id': 'Sci-Fi', 'movieCount': 958, 'averageRating': 6.12}
...
```

In [19]:
# Exercise 5: Calculate the Average IMDb Rating and Number of Movies for Each Genre (Rounded to 2 Decimals)

pipeline5 = [
    # BEGIN SOLUTION
    {
        "$unwind": "$genres"
    },
    {
        "$group": {
            "_id": "$genres",
            "averageRating": { "$avg": "$imdb.rating" },
            "movieCount": { "$sum": 1 }
        }
    },
    {
        "$project": {
            "_id": 1,
            "averageRating": { "$round": ["$averageRating", 2] },
            "movieCount": 1
        }
    }
    # END SOLUTION
]

# Execute the pipeline
result5 = list(collection.aggregate(pipeline5))
for doc in result5:
    print(doc)

{'_id': 'Drama', 'movieCount': 12385, 'averageRating': 6.8}
{'_id': 'Action', 'movieCount': 2381, 'averageRating': 6.35}
{'_id': 'Horror', 'movieCount': 1470, 'averageRating': 5.78}
{'_id': 'Animation', 'movieCount': 912, 'averageRating': 6.9}
{'_id': 'Romance', 'movieCount': 3318, 'averageRating': 6.66}
{'_id': 'History', 'movieCount': 874, 'averageRating': 7.17}
{'_id': 'War', 'movieCount': 699, 'averageRating': 7.13}
{'_id': 'Documentary', 'movieCount': 1834, 'averageRating': 7.37}
{'_id': 'Musical', 'movieCount': 440, 'averageRating': 6.67}
{'_id': 'Biography', 'movieCount': 1269, 'averageRating': 7.09}
{'_id': 'Film-Noir', 'movieCount': 77, 'averageRating': 7.4}
{'_id': 'Talk-Show', 'movieCount': 1, 'averageRating': 7.0}
{'_id': 'Fantasy', 'movieCount': 1055, 'averageRating': 6.38}
{'_id': 'Mystery', 'movieCount': 1139, 'averageRating': 6.53}
{'_id': 'Crime', 'movieCount': 2457, 'averageRating': 6.69}
{'_id': 'Thriller', 'movieCount': 2454, 'averageRating': 6.3}
{'_id': 'Adventure

In [20]:
# Filter for the genre 'Adventure' and extract the document
adventure_doc = list(filter(lambda x: x['_id'] == 'Adventure', result5))[0]

# Assert the movie count for 'Adventure'
assert adventure_doc['movieCount'] == 1900, f"expected movieCount 1900 for 'Adventure' but got {adventure_doc['movieCount']}"

# Assert the average rating for 'Adventure'
assert adventure_doc['averageRating'] == 6.49, f"expected averageRating 6.49 for 'Adventure' but got {adventure_doc['averageRating']}"

### Exercise 6: Implementing ACID Transactions for a Shopping Cart

rubric: {accuracy = 4}

In this exercise, you will implement an ACID (Atomicity, Consistency, Isolation, Durability) transaction using MongoDB. The goal is to ensure that a series of operations either all succeed or all fail, maintaining the integrity of the database.

#### Instructions

1. **Setup MongoDB Client**: Ensure you have the MongoDB client installed and connected to your database.
2. **Create Collections**: Create two collections in your MongoDB database: `inventory` and `carts`.
3. **Insert Sample Data**:
    - Insert sample items into the `inventory` collection with fields `item_id` and `quantity`.
    - Insert a sample user into the `carts` collection with fields `user_id` and `items`.
4. **Implement the Transaction**:
    - Write a function `add_to_cart` that performs the following operations within a transaction:
        - Check if the inventory has enough quantity for the specified item.
        - Deduct the quantity from the inventory.
        - Add the item to the user's shopping cart.
    - Ensure that if any operation fails, the transaction is aborted and no changes are made to the database.
5. **Run the Transaction**: Execute the transaction and handle any errors that may occur.




In [21]:
from pymongo.errors import ConnectionFailure, OperationFailure
from pymongo.write_concern import WriteConcern
from pymongo.read_concern import ReadConcern
from pymongo.read_preferences import ReadPreference

client.drop_database('shop')  # drop the shop database if it exists

# Select the database and collections
db = client['shop']
inventory = db['inventory']
carts = db['carts']

# Insert sample data into the inventory collection
inventory.insert_many([
    {'item_id': 'item456', 'quantity': 10},
    {'item_id': 'item789', 'quantity': 5},
    {'item_id': 'item123', 'quantity': 20}
])

# Insert sample data into the carts collection
carts.insert_one({
    'user_id': 'user123',
    'items': []
})


InsertOneResult(ObjectId('66f43db2de4414a6c3e82b59'), acknowledged=True)

#### 6.1 Write a function to check if the inventory has enough quantity for a particular item
- raise ValueError if the order_quantity exceed the inventory quantity
- Print out a message `Item {item_id} has enough quantity` if the order_quantity <= the inventory quantity

In [22]:
def check_quantity(item_id, order_quantity):
    # BEGIN SOLUTION
    # Find the item in the inventory collection
    item = inventory.find_one({'item_id': item_id})

    # Check if the item exists and has enough quantity
    if item and item['quantity'] < order_quantity:
        raise ValueError(f"Not enough inventory for item_id {item_id}")
    else:
        print(f"Item {item_id} has enough quantity")
    # END SOLUTION


In [23]:
# print out the quantity of item123 from the inventory collection
print(inventory.find_one({'item_id': 'item123'}))


{'_id': ObjectId('66f43db2de4414a6c3e82b58'), 'item_id': 'item123', 'quantity': 20}


In [24]:
check_quantity('item123', 10) # this should print out a message that item123 has enough quantity

Item item123 has enough quantity


In [25]:
check_quantity('item123', 100) # this should raise a ValueError

ValueError: Not enough inventory for item_id item123

#### 6.2 Write a function to update the quantity from the inventory

- Use the `update_one` function to update the quantity of inventory
- You can use the `$inc` to increment the quantity of inventory based on the order quantity

In [26]:
def update_quantity(order_quantity, item_id):
    # BEGIN SOLUTION
    inventory.update_one(
        {'item_id': item_id},
        {'$inc': {'quantity': -order_quantity}}
    )
    # END SOLUTION


In [27]:
# print out the quantity of item_id 'item123' before updating
print(inventory.find_one({'item_id': 'item123'}))

{'_id': ObjectId('66f43db2de4414a6c3e82b58'), 'item_id': 'item123', 'quantity': 20}


In [28]:

# Update the quantity for item_id 'item123'
order_quantity = 10
item_id = 'item123'
update_quantity(order_quantity, item_id)

In [29]:
# print out the quantity of item_id 'item123' after updating
print(inventory.find_one({'item_id': 'item123'}))

{'_id': ObjectId('66f43db2de4414a6c3e82b58'), 'item_id': 'item123', 'quantity': 10}


#### 6.3 Write a function to update user's shopping cart

- Use the `update_one` function to update the shopping cart
- Use the `$push` operator to an element to an array. If the array does not exist, it will be created.

In [30]:
# print out shopping cart before adding items
print(carts.find_one({'user_id': 'user123'}))

{'_id': ObjectId('66f43db2de4414a6c3e82b59'), 'user_id': 'user123', 'items': []}


In [31]:
# write a function to add items to the shopping cart
def add_to_cart(user_id, item_id, quantity):
    # BEGIN SOLUTION
    carts.update_one(
        {'user_id': user_id},
        {'$push': {'items': {'item_id': item_id, 'quantity': quantity}}}
    )
    # END SOLUTION

In [32]:
# test the function by adding items to the shopping cart
add_to_cart('user123', 'item123', 5)

In [33]:
# print out shopping cart after adding items
print(carts.find_one({'user_id': 'user123'}))

{'_id': ObjectId('66f43db2de4414a6c3e82b59'), 'user_id': 'user123', 'items': [{'item_id': 'item123', 'quantity': 5}]}


#### 6.4 Combine everything into a transaction function

Create a transaction function that:
- First check if the quantity is enough in the inventory, if not, it should raise a ValueError
- Update the quantity from the inventory by substracting the order_quantity
- Update user's shopping cart with the item_id and the order_quantity

Note: You need to add a line `session=session` in the update_one function for inventory and shopping carts

In [34]:

# Define the transaction function
def add_to_cart(session, user_id, item_id, quantity):
    # Check if the inventory has enough quantity
    # Deduct the quantity from the inventory 
    # Add the item to the user's shopping cart
          
    # BEGIN SOLUTION
    # Check if the inventory has enough quantity
    item = inventory.find_one({'item_id': item_id}, session=session)
    if item is None or item['quantity'] < quantity:
        raise ValueError(f"Not enough inventory for item_id {item_id}")

    # Deduct the quantity from the inventory  

    inventory.update_one(
        {'item_id': item_id},
        {'$inc': {'quantity': -quantity}},
        session=session
    )
    
    # Add the item to the user's shopping cart
    carts.update_one(
        {'user_id': user_id},
        {'$push': {'items': {'item_id': item_id, 'quantity': quantity}}},
        upsert=True,
        session=session
    )
    # END SOLUTION


In [35]:

# Start a session
with client.start_session() as session:
    try:
        # Start a transaction
        session.start_transaction()

        # Perform the add to cart operation
        add_to_cart(session, 'user123', 'item456', 2)

        # Commit the transaction
        session.commit_transaction()
        print("Transaction committed.")
    except (ConnectionFailure, OperationFailure, ValueError) as e:
        # Abort the transaction on error
        session.abort_transaction()
        print(f"Transaction aborted due to error: {e}")

Transaction committed.


Let's test the constraint of the transaction. This should throws an error because order_quantity of 200 is exceeding the inventory quantity

In [36]:

# Start a session
with client.start_session() as session:
    try:
        # Start a transaction
        session.start_transaction()

        # Perform the add to cart operation
        add_to_cart(session, 'user123', 'item456', 200)

        # Commit the transaction
        session.commit_transaction()
        print("Transaction committed.")
    except (ConnectionFailure, OperationFailure, ValueError) as e:
        # Abort the transaction on error
        session.abort_transaction()
        print(f"Transaction aborted due to error: {e}")

Transaction aborted due to error: Not enough inventory for item_id item456


**There are no auto-tests for Q6, it will be graded manually**

## Submission instructions

{rubric: mechanics = 5}

- Make sure the notebook can run from top to bottom without any error. Restart the kernel and run all cells.
- Commit and push your notebook to the github repo
- Double check your notebook is rendered properly on Github and you can see all the outputs clearly